In [5]:
import pandas as pd
import numpy as np
import warnings
from process import process_and_save_trans, process_zenmoney_export, UNKNOWN_CATEGORY

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.0f}'.format

In [2]:
%%time

filename = './zen_2021-05-28_dumpof_transactions_from_alltime.csv'
# process_and_save_trans(filename)
df = process_zenmoney_export(filename)
# df.head()

Wall time: 1.05 s


In [6]:
def add_ym(df):
    ym_df = pd.read_excel('./out_template.xlsx', sheet_name='budget', index=False)
    ym_df.sub_category.fillna(UNKNOWN_CATEGORY, inplace=True)

    df['ym'] = None
    df['lim'] = 0

    sub_category_mask = ym_df.sub_category != UNKNOWN_CATEGORY
    
    # set ym for all limits with sub_category
    ym_sub_df = ym_df[sub_category_mask]
    for _, row in ym_sub_df.iterrows():
        apply_mask = ((df.pay_type == row.pay_type) & (df.category == row.category)) & (df.sub_category == row.sub_category)
        df.loc[apply_mask, 'ym'] = row.ym
        df.loc[apply_mask, 'lim'] = row.lim

    # set (!) unfilled ym for others
    ym_no_sub_df = ym_df[~sub_category_mask]
    for _, row in ym_no_sub_df.iterrows():
        apply_mask = pd.isnull(df.ym) & ((df.pay_type == row.pay_type) & (df.category == row.category))
        df.loc[apply_mask, 'ym'] = row.ym
        df.loc[apply_mask, 'lim'] = row.lim

    tmp_df = df[pd.isnull(df.ym)]
    unknown_ym_set = set(zip(tmp_df.pay_type, tmp_df.category, tmp_df.sub_category))
    assert not unknown_ym_set, f'Found categories with unkown limit type: {unknown_ym_set} '

    # remove subcategory for all annual items without sub categories
    # for _, row in ym_df[(ym_df.ym == 'year') & (~sub_category_mask)]
    
    return df

In [11]:
agg_df = df.copy()
agg_df.dt = agg_df.dt.astype('datetime64[M]')

agg_df = agg_df.groupby(['pay_type', 'category', 'sub_category', 'dt'], as_index=False).amount_rub.sum()

agg_df = add_ym(agg_df)

month_df = agg_df[agg_df.ym == 'month']
start_date = pd.to_datetime('2021-01-01')
end_date = pd.to_datetime('2021-12-31')
month_df = month_df.query('dt >= @start_date and dt <= @end_date')
# month_df.dt = month_df.dt.astype('datetime64[Y]')
month_df = month_df.groupby(['pay_type', 'category', 'dt'], as_index=False) \
    .aggregate({'amount_rub': 'sum', 'lim': 'min'})

year_df = agg_df[agg_df.ym == 'year']
year_df.dt = year_df.dt.astype('datetime64[Y]')
year_df = year_df.groupby(['pay_type', 'category', 'sub_category', 'dt'], as_index=False) \
    .aggregate({'amount_rub': 'sum', 'lim': 'min'})

# agg_df.head()

# month_agg

# agg_df.pivot_table(index=['pay_type', 'ym', 'category',], columns=['dt'], values='amount_rub', aggfunc='sum')

piv_month_df = month_df.pivot_table(index=['pay_type', 'category'], columns=['dt'], values='amount_rub', aggfunc='sum')
# piv_month_df['avg'] = piv_month_df / 12

piv_month_df

dt                         2021-01-01  2021-02-01  2021-03-01  2021-04-01
pay_type category                                                        
expense  А ежемесячно          36,481     119,668      67,081      28,290
         Алкоголь               4,700       6,506       9,264         nan
         Витя                     nan         nan       6,900         nan
         Ира                   11,830      15,410      35,309         664
         Коммуналка            13,692       8,567      65,393       2,166
         Машина                 5,168      11,094       2,650         nan
         Помощь                61,250      48,400      52,500      44,250
         Супермаркет           34,309      50,555      35,359      15,883
         Ю ежемесячно          68,140      39,472      59,523      28,459
profit   А ежемесячно           3,081      43,324      25,200       2,400
         Авито                    nan       1,500         nan         nan
         Аня психология           nan       3,500      12,000         nan
         Аня фото и видео         127      12,566         405         nan
         Вычет                    nan         nan     135,200         nan
         Зарплата Ю сбер       75,600     219,240     219,240     143,640
         Корректировка            nan         nan       1,583         nan
         Подарки доход         10,000       2,000         nan         nan
         Проценты                 149         205          25         nan
         Прочее                   nan         nan         263         nan

In [16]:
month_df['util_pcnt'] = (month_df.amount_rub / month_df.lim).round(2) * 100
month_df.head()
unpiv_month_df = month_df.melt(id_vars=('pay_type', 'category', 'dt'), var_name='measure', value_name='value')
unpiv_month_df.pivot_table(index=['pay_type', 'category'], columns=['dt', 'measure'], values='value', aggfunc='sum')

dt                        2021-01-01                   2021-02-01          \
measure                   amount_rub     lim util_pcnt amount_rub     lim   
pay_type category                                                           
expense  А ежемесячно         36,481  40,000        91    119,668  40,000   
         Алкоголь              4,700  10,000        47      6,506  10,000   
         Витя                    nan     nan       nan        nan     nan   
         Ира                  11,830  10,000       118     15,410  10,000   
         Коммуналка           13,692  20,000        68      8,567  20,000   
         Машина                5,168  10,000        52     11,094  10,000   
         Помощь               61,250  60,000       102     48,400  60,000   
         Супермаркет          34,309  40,000        86     50,555  40,000   
         Ю ежемесячно         68,140  40,000       170     39,472  40,000   
profit   А ежемесячно          3,081  25,000        12     43,324  25,000   
         Авито                   nan     nan       nan      1,500       0   
         Аня психология          nan     nan       nan      3,500       0   
         Аня фото и видео        127       0       inf     12,566       0   
         Вычет                   nan     nan       nan        nan     nan   
         Зарплата Ю сбер      75,600 292,320        26    219,240 292,320   
         Корректировка           nan     nan       nan        nan     nan   
         Подарки доход        10,000       0       inf      2,000       0   
         Проценты                149  25,000         1        205  25,000   
         Прочее                  nan     nan       nan        nan     nan   

dt                                  2021-03-01                   2021-04-01  \
measure                   util_pcnt amount_rub     lim util_pcnt amount_rub   
pay_type category                                                             
expense  А ежемесячно           299     67,081  40,000       168     28,290   
         Алкоголь                65      9,264  10,000        93        nan   
         Витя                   nan      6,900  10,000        69        nan   
         Ира                    154     35,309  10,000       353        664   
         Коммуналка              43     65,393  20,000       327      2,166   
         Машина                 111      2,650  10,000        26        nan   
         Помощь                  81     52,500  60,000        88     44,250   
         Супермаркет            126     35,359  40,000        88     15,883   
         Ю ежемесячно            99     59,523  40,000       149     28,459   
profit   А ежемесячно           173     25,200  25,000       101      2,400   
         Авито                  inf        nan     nan       nan        nan   
         Аня психология         inf     12,000       0       inf        nan   
         Аня фото и видео       inf        405       0       inf        nan   
         Вычет                  nan    135,200       0       inf        nan   
         Зарплата Ю сбер         75    219,240 292,320        75    143,640   
         Корректировка          nan      1,583       0       inf        nan   
         Подарки доход          inf        nan     nan       nan        nan   
         Проценты                 1         25  25,000         0        nan   
         Прочее                 nan        263       0       inf        nan   

dt                                           
measure                       lim util_pcnt  
pay_type category                            
expense  А ежемесячно      40,000        71  
         Алкоголь             nan       nan  
         Витя                 nan       nan  
         Ира               10,000         7  
         Коммуналка        20,000        11  
         Машина               nan       nan  
         Помощь            60,000        74  
         Супермаркет       40,000        40  
         Ю ежемесячно      40,000        71  
profit   А еже

In [84]:
year_df.pivot_table(index=['pay_type', 'category', 'sub_category'], columns=['dt'], values='amount_rub', aggfunc='sum')

dt                                                         2018-01-01  \
pay_type category              sub_category                             
expense  Коммуналка            Налог квартиры                     nan   
                               Страховка Дыбенко                  nan   
                               Страховка Илимская                 nan   
         Машина                Налог машина                       nan   
                               Ремонт                             nan   
                               Страховка Авто                     nan   
         Медицина              unknown                            nan   
                               Медицина А                         nan   
                               Медицина В                         nan   
                               Медицина И                         nan   
                               Медицина Ю                         nan   
         Особо крупные расходы unknown                            nan   
                               Образование А                      nan   
                               Ю зубы                             nan   
         Путешествия           skipass аренда лыж и школа         nan   
                               unknown                            nan   
                               Аренда машины                      nan   
                               Визы и документы                   nan   
                               Лыжи                               nan   
                               Перелет и трансфер              69,200   
                               Проживание                      57,000   
                               Развлечения                        nan   
         Ремонт и техника      unknown                            nan   
                               Ремонт                             nan   
                               Техника                            nan   

dt                                                         2019-01-01  \
pay_type category              sub_category                             
expense  Коммуналка            Налог квартиры                  13,844   
                               Страховка Дыбенко               20,500   
                               Страховка Илимская                 nan   
         Машина                Налог машина                    17,000   
                               Ремонт                         256,440   
                               Страховка Авто                  64,600   
         Медицина              unknown                          4,727   
                               Медицина А                      18,720   
                               Медицина В                      11,250   
                               Медицина И                      21,715   
                               Медицина Ю                       7,337   
         Особо крупные расходы unknown                         73,541   
                               Образование А                      nan   
                               Ю зубы                             nan   
         Путешествия           skipass аренда лыж и школа      90,650   
                               unknown                            nan   
                               Аренда машины                  121,822   
                               Визы и документы                 9,100   
                               Лыжи                               nan   
                               Перелет и трансфер             177,334   
                               Проживание                     675,886   
                               Развлечения                    357,838   
         Ремонт и техника      unknown                          3,850   
                               Ремонт                           2,847   
                               Техника                         12,256   

dt                                                      

In [86]:
year_df.pivot_table(index=['pay_type', 'category'], columns=['dt'], values='amount_rub', aggfunc='sum')

dt                              2018-01-01  2019-01-01  2020-01-01  2021-01-01
pay_type category                                                             
expense  Коммуналка                    nan      34,344      34,063      23,044
         Машина                        nan     338,040     218,870      99,297
         Медицина                      nan      63,749     182,149      62,535
         Особо крупные расходы         nan      73,541     643,082     227,297
         Путешествия               126,200   1,432,629     584,007     323,974
         Ремонт и техника              nan      18,953     136,402      49,042

In [71]:


start_date = pd.to_datetime('2020-01-01')
end_date = pd.to_datetime('2020-12-31')

exp_df = df.query('pay_type=="expense" and dt >= @start_date and dt <= @end_date')
month_exp_df = exp_df.query('ym == "month"')
month_exp_df.loc[:, 'dt'] = month_exp_df.dt.astype('datetime64[M]')
month_exp_df.pivot_table(index=['category'], columns=['dt'], values='amount_rub', aggfunc='sum')


C:\Apps\Anaconda\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


dt,2020-01-01 00:00:00,2020-02-01 00:00:00,2020-03-01 00:00:00,2020-04-01 00:00:00,2020-05-01 00:00:00,2020-06-01 00:00:00,2020-07-01 00:00:00,2020-08-01 00:00:00,2020-09-01 00:00:00,2020-10-01 00:00:00,2020-11-01 00:00:00,2020-12-01 00:00:00
category,,,,,,,,,,,,
А ежемесячно,"87,925","26,128","14,382","9,475","19,775","73,463","30,681","17,825","73,694","21,429","44,035","46,860"
Алкоголь,"1,200","3,810","22,399","3,420","5,860","3,667","7,542","14,268","10,228","4,669","9,467","16,180"
Витя,"3,803",nan,nan,nan,720,nan,"5,000",400,"3,535",nan,382,"9,495"
Ира,"2,647","7,600",nan,"5,620","11,500","21,967","7,300","11,200","10,565","10,839","9,205","9,000"
Коммуналка,"31,025","6,458","40,432","22,712","4,391","3,969","34,983","4,569","52,858","3,369","2,869","29,500"
Корректировка,741,150,1,nan,nan,nan,118,nan,nan,nan,nan,nan
Машина,"4,912","3,273","6,740",505,"2,608","14,085","11,589","13,627","11,700","6,849","8,253","10,037"
Помощь,"40,100","46,132","57,650","60,500","57,000","57,250","60,300","56,000","54,781","71,300","69,800","56,000"
Супермаркет,"29,692","25,917","59,167","48,328","49,408","51,515","46,987","44,152","58,589","37,000","40,447","54,658"


In [21]:
exp_df.dt = exp_df.dt.astype('datetime64[M]')
exp_df.pivot_table(index=['category', 'sub_category'], columns=['dt'], values='amount_rub', aggfunc='sum')

dt                                   2020-01-01  2020-02-01  2020-03-01  \
category         sub_category                                             
А ежемесячно     unknown                 19,395       6,650       5,598   
                 А спорт                 25,636         nan         nan   
                 Дача                       nan         nan         nan   
                 Обучение А                 nan         nan         nan   
                 Одежда А                 3,300         nan         nan   
                 Подарки А                6,892       7,468         nan   
                 Развлечения А           28,129       9,220       8,214   
                 Рестораны А              2,390       2,290         nan   
                 Такси А                  1,491         nan         570   
                 Транспорт А                692         500         nan   
Алкоголь         unknown                  1,200       3,810      22,399   
Витя             В занятия                  nan         nan         nan   
                 В одежда и обувь         3,803         nan         nan   
                 В развлечения              nan         nan         nan   
Ира              И занятия                1,020       6,600         nan   
                 И одежда и обувь         1,627       1,000         nan   
                 И развлечения              nan         nan         nan   
Коммуналка       unknown                    nan         nan       1,200   
                 ЖКХ                     23,228         nan      34,782   
                 Интернет                   nan         nan       1,780   
                 Налог квартиры             nan         nan         nan   
                 Подписки                   169       4,568         169   
                 Сотовые                  7,628       1,890       2,500   
                 Страховка Дыбенко       20,862         nan         nan   
                 Страховка Илимская       4,950         nan         nan   
Корректировка    unknown                    741         150           1   
Машина           Бензин                   3,704       2,573       5,512   
                 Мойка                      350         700         nan   
                 Налог машина               nan         nan         nan   
                 Парковка                   605         nan         975   
...                                         ...         ...         ...   
Путешествия      Проживание              14,088      58,979         nan   
                 Развлечения             23,963     149,407      33,615   
Ремонт и техника unknown                    nan         nan         nan   
                 Ремонт                     nan         nan         nan   
                 Техника                    nan      14,700         nan   
Супермаркет      10очка                   4,294       2,599       3,160   
                 5ка                        622         nan       1,690   
                 76                         976       1,859         273   
                 Spar                       nan         nan         nan   
                 unknown                  6,458         100       1,106   
                 Ашан                     6,615       4,981       7,965   
                 Вкусвилл                 3,767       5,007       8,878   
                 Кофе                       872       1,210       1,718   
                 Магнит                     164         295       1,135   
                 Метро                    3,742       4,689      19,423   
                 Мираторг                   nan         nan         nan   
                 Перекресток              2,181       1,330      13,464   
                 Фрукты                     nan       3,847         355   
Ю ежемесячно     unknown                  1,272       4,700         120   
                 Велосипед Ю                nan         nan         nan   
                 Игры    

In [12]:
year_exp_df = exp_df.query('ym == "year"')
year_exp_df.dt = year_exp_df.dt.astype('datetime64[Y]')
year_exp_df.pivot_table(index=['category'], columns=['dt'], values='amount_rub', aggfunc='sum')


dt,2020-01-01 00:00:00
category,
Коммуналка,"34,063"
Машина,"218,870"
Медицина,"64,501"
Особо крупные расходы,"99,630"
Ремонт и техника,"5,125"
